<a href="https://colab.research.google.com/github/itsjacobjoy/Prompt-Classification/blob/main/Prompt_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Prompt** **Classification**

In [1]:
!pip3 install --upgrade --user google-cloud-aiplatform rouge_score plotly


In [2]:
#Authenticating colab for access to google cloud
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

**Importing necessary libraries**

In [84]:
import time
import base64
from typing import Dict, List

# For data handling.
import pandas as pd

# For visualization.
import plotly.graph_objects as go

# For fine tuning Gemini model.
import vertexai
import vertexai.preview.generative_models as generative_models
from vertexai.preview import tuning

# For extracting vertex experiment details.
from google.cloud import aiplatform
from google.cloud.aiplatform.metadata import context
from google.cloud.aiplatform.metadata import utils as metadata_utils
from plotly.subplots import make_subplots

# For evaluation metric computation.
from tqdm import tqdm
from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    Part,
    HarmBlockThreshold,
    HarmCategory,
)
from vertexai.preview.tuning import sft



**Initializing Project**

In [4]:
PROJECT_ID = "prompt-categorization"  # @param
LOCATION = "us-central1"  # @param

vertexai.init(project=PROJECT_ID, location=LOCATION)

**Create a tuning job and training Model**

In [5]:
# Dataset for model tuning.
training_data_path = "gs://training_data_prompt_categorization-7735/training_data_4.jsonl"
# Dataset for model evaluation.
validation_data_path = "gs://training_data_prompt_categorization-7735/validation_data.jsonl"

In [ ]:
sft_tuning_job = sft.train(
    source_model="gemini-1.0-pro-002",
    train_dataset=training_data_path,
    validation_dataset=validation_data_path,
    epochs=4,
    adapter_size=4,
    learning_rate_multiplier=0.2,
    tuned_model_display_name="prompt_categorization_tuned_gemini_pro_2.1",
)

sft_tuning_job.to_dict()

INFO:vertexai.tuning._tuning:Creating SupervisedTuningJob
INFO:vertexai.tuning._tuning:SupervisedTuningJob created. Resource name: projects/651526622405/locations/us-central1/tuningJobs/50154944455507968
INFO:vertexai.tuning._tuning:To use this SupervisedTuningJob in another session:
INFO:vertexai.tuning._tuning:tuning_job = sft.SupervisedTuningJob('projects/651526622405/locations/us-central1/tuningJobs/50154944455507968')
INFO:vertexai.tuning._tuning:View Tuning Job:
https://console.cloud.google.com/vertex-ai/generative/language/locations/us-central1/tuning/tuningJob/50154944455507968?project=651526622405


{'name': 'projects/651526622405/locations/us-central1/tuningJobs/50154944455507968',
 'tunedModelDisplayName': 'prompt_categorization_tuned_gemini_pro_2.1',
 'baseModel': 'gemini-1.0-pro-002',
 'supervisedTuningSpec': {'trainingDatasetUri': 'gs://training_data_prompt_categorization-7735/training_data_4.jsonl',
  'validationDatasetUri': 'gs://training_data_prompt_categorization-7735/validation_data.jsonl',
  'hyperParameters': {'epochCount': '4',
   'learningRateMultiplier': 0.2,
   'adapterSize': 'ADAPTER_SIZE_FOUR'}},
 'state': 'JOB_STATE_PENDING',
 'createTime': '2024-08-02T10:01:21.471814Z',
 'updateTime': '2024-08-02T10:01:21.471814Z'}

In [ ]:
# Refreshing each time until job is complete
while not sft_tuning_job.has_ended:
    time.sleep(60)
    sft_tuning_job.refresh()

print(sft_tuning_job.tuned_model_name)
print(sft_tuning_job.tuned_model_endpoint_name)
print(sft_tuning_job.experiment)

**Loading Tuned Model**

In [88]:
# Getting list of tuned models
responses = sft.SupervisedTuningJob.list()

for response in responses:
    print(response)

resource name: projects/651526622405/locations/us-central1/tuningJobs/50154944455507968
resource name: projects/651526622405/locations/us-central1/tuningJobs/8046190529735557120
resource name: projects/651526622405/locations/us-central1/tuningJobs/7558086531840212992
resource name: projects/651526622405/locations/us-central1/tuningJobs/1703969966211989504
resource name: projects/651526622405/locations/us-central1/tuningJobs/7864612781478117376


In [81]:
#Load the model
sft_tuning_job = sft.SupervisedTuningJob("projects/651526622405/locations/us-central1/tuningJobs/50154944455507968")

**Getting Evaluation Metrics**

In [49]:
experiment_name = sft_tuning_job.experiment.resource_name
experiment_name

'projects/651526622405/locations/us-central1/metadataStores/default/contexts/tuning-experiment-20240802030927695460'

In [50]:
# Locate Vertex AI Experiment and Vertex AI Experiment Run
experiment = aiplatform.Experiment(experiment_name=experiment_name)

filter_str = metadata_utils._make_filter_string(
    schema_title="system.ExperimentRun",
    parent_contexts=[experiment.resource_name],
)

experiment_run = context.Context.list(filter_str)[0]

print("Experiment name: ", experiment.name)
print("Experiment Run name: ", experiment_run.display_name)

Experiment name:  tuning-experiment-20240802030927695460
Experiment Run name:  tuning-experiment-run-20240802030927695460


In [51]:
# Read data from Tensorboard
tensorboard_run_name = f"{experiment.get_backing_tensorboard_resource().resource_name}/experiments/{experiment.name}/runs/{experiment_run.display_name}"
tensorboard_run = aiplatform.TensorboardRun(tensorboard_run_name)
metrics = tensorboard_run.read_time_series_data()

In [52]:
def get_metrics(metric: str = "/train_total_loss"):
    """
    Get metrics from Tensorboard.

    Args:
      metric: metric name, eg. /train_total_loss or /eval_total_loss.
    Returns:
      steps: list of steps.
      steps_loss: list of loss values.
    """
    loss_values = metrics[metric].values
    steps_loss = []
    steps = []
    for loss in loss_values:
        steps_loss.append(loss.scalar.value)
        steps.append(loss.step)
    return steps, steps_loss

In [53]:
# Get Train and Eval Loss
train_loss = get_metrics(metric="/train_total_loss")
eval_loss = get_metrics(metric="/eval_total_loss")

In [69]:
train_correct_preds = get_metrics(metric="/train_fraction_of_correct_next_step_preds")
eval_correct_preds = get_metrics(metric="/eval_fraction_of_correct_next_step_preds")


**Plotting Evaluation Metrics**

In [54]:
fig = make_subplots(
    rows=1, cols=2, shared_xaxes=True, subplot_titles=("Train Loss", "Eval Loss")
)

# Add traces
fig.add_trace(
    go.Scatter(x=train_loss[0], y=train_loss[1], name="Train Loss", mode="lines"),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(x=eval_loss[0], y=eval_loss[1], name="Eval Loss", mode="lines"),
    row=1,
    col=2,
)

# Add figure title
fig.update_layout(title="Train and Eval Loss", xaxis_title="Steps", yaxis_title="Loss")

# Set x-axis title
fig.update_xaxes(title_text="Steps")

# Set y-axes titles
fig.update_yaxes(title_text="Loss")

# Show plot
fig.show()

In [72]:
fig = make_subplots(
    rows=1, cols=2, shared_xaxes=True, subplot_titles=("Train Correct Preds", "Eval Correct Preds")
)

# Add traces
fig.add_trace(
    go.Scatter(x=train_correct_preds[0], y=train_correct_preds[1], name="Train Correct Preds", mode="lines"),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(x=eval_correct_preds[0], y=eval_correct_preds[1], name="Eval Correct Preds", mode="lines"),
    row=1,
    col=2,
)

# Add figure title
fig.update_layout(title="Train and Eval Fraction of Correct Predictions", xaxis_title="Steps", yaxis_title="Value")

# Set x-axis title
fig.update_xaxes(title_text="Steps")

# Set y-axes titles
fig.update_yaxes(title_text="Value")

# Show plot
fig.show()

**Inferencing the Model**


In [26]:
tuned_model_endpoint_name = "projects/651526622405/locations/us-central1/endpoints/9199831492904091648"
tuned_model_endpoint_name

'projects/651526622405/locations/us-central1/endpoints/9199831492904091648'

In [86]:
def multiturn_generate_content(prompt):
  vertexai.init(project="651526622405", location="us-central1") # Initializing model

  # Extracting tuned model
  model = GenerativeModel(
    tuned_model_endpoint_name,
    system_instruction=[system_inst]
  )


  chat = model.start_chat()

  # Inferencing model
  response = chat.send_message(
      [prompt],
      generation_config=generation_config,
      safety_settings=safety_settings
  )

  result = response.text  # Extract and print only the text content

  # If the result is in the "others category ---> Route to general llm"
  if result=='Others':
    generation_model = GenerativeModel("gemini-1.0-pro-002")

    response = generation_model.generate_content(contents=prompt)
    print(f"Category: {result}")
    print("Response: ", response.text)

  else:
      print("Response:", result)



**Example Outputs**

In [85]:
prompt = """Can you check and correct this snippet of code"""

# System intruction to tuned model so that it gives a correct output
system_inst = """You should classify the text into one of the following classes:[Chatbots and Virtual Assistants, Conversation, Mental health, Image Generation, Video Generation, Music Creation, Speech Generation, Podcast Content Creation, Blog Writing, Podcasts, Storytelling and Narrative Creation, Social Media Posts, Product Descriptions, SEO Optimization, Market Analysis, Translation, Coding and Programming Assistance, API Integration, Data Analysis, Financial Report Generation, Educational Content Creation, Language Learning Assistance, Presentation Creation, Email Generation, Legal Document Drafting, Contract Generation, Recipe Generation, Fitness and Health Plans Recommendations, Homework Solutions, AI content detection, Branding and Logo Design, Travel Itinerary Creation, Event Planning, Personalized Recommendations, Resume and Cover Letter Writing, Job Description Creation, Research Paper Data, Summarisations, News Article Generation, Content Summarization, Others]

only respond with the classes I have listed above. Do not give another category or a similar class as a response"""

generation_config = {
    "max_output_tokens": 1024,
    "temperature": 1,
    "top_p": 1,
}

safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}

#Call function to inference model
multiturn_generate_content(prompt)

Response: Coding and Programming Assistance


In [87]:
prompt = """Whats the weather like today?"""

system_inst = """You should classify the text into one of the following classes:[Chatbots and Virtual Assistants, Conversation, Mental health, Image Generation, Video Generation, Music Creation, Speech Generation, Podcast Content Creation, Blog Writing, Podcasts, Storytelling and Narrative Creation, Social Media Posts, Product Descriptions, SEO Optimization, Market Analysis, Translation, Coding and Programming Assistance, API Integration, Data Analysis, Financial Report Generation, Educational Content Creation, Language Learning Assistance, Presentation Creation, Email Generation, Legal Document Drafting, Contract Generation, Recipe Generation, Fitness and Health Plans Recommendations, Homework Solutions, AI content detection, Branding and Logo Design, Travel Itinerary Creation, Event Planning, Personalized Recommendations, Resume and Cover Letter Writing, Job Description Creation, Research Paper Data, Summarisations, News Article Generation, Content Summarization, Others]

only respond with the classes I have listed above. Do not give another category or a similar class as a response"""

generation_config = {
    "max_output_tokens": 1024,
    "temperature": 1,
    "top_p": 1,
}

safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}

multiturn_generate_content(prompt)

Category: Others
Response:  As a large language model, I don't have access to real-time information like the current weather. To get an accurate weather update for today, I recommend checking a reliable weather app or website. You can specify your location to get a more precise forecast. 

Would you like me to help you find a weather app or website for your location? 

